In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [4]:
crash=pd.read_csv('dataset\data_200.csv')

How many NaN values?

In [5]:
crash.isnull().sum().sum()

446

Using geopy to complete missing values in the columns **borough** and **zip_code** 

In [6]:
geolocator = Nominatim(user_agent="essay")

In [7]:
crash.location=crash.location.apply(lambda x: str(x).replace('(','').replace(')',''))

In [8]:
crash.location=crash.location.replace('nan', np.nan)

In [9]:
crash.location.fillna("0,0", inplace=True)

In [10]:
a = crash[crash.borough.isnull()]

In [11]:
atwo = crash[crash.zip_code.isnull()]

To avoid looking for all the coordiantes, i will create a group of uniques values, in case some coordiantes are repeated.

In [12]:
b=a.location.unique()

In [13]:
btwo=atwo.location.unique()

In [14]:
new_b=b[b!='0,0']

In [15]:
new_btwo=btwo[btwo!='0,0']

extract the name of the suburb and post code. 

In [47]:
# def find_suburb(x):
#     try:
#         sub=x.raw['address']['suburb']
#     except:
#         pass
#     return sub



In [17]:
c = dict(zip(new_b, pd.Series(new_b).apply(geolocator.reverse).apply(lambda x: x.raw['address']['suburb'])))

In [19]:
ctwo = dict(zip(new_b, pd.Series(new_b).apply(geolocator.reverse).apply(lambda x: x.raw['address']['postcode'])))

create two columns with the extraxted information

In [20]:
crash['missing_suburb'] = crash['location'].map(c)

In [21]:
crash['missing_postcode'] = crash['location'].map(ctwo)

In [22]:
crash.missing_suburb=crash.missing_suburb.apply(lambda x: (str(x).replace('The ','')).upper() if type(x)== str else x)

complete missign values with the info extracted with geopy

In [23]:
crash.borough.fillna(crash.missing_suburb, inplace=True)

In [24]:
crash.zip_code.fillna(crash.missing_postcode, inplace=True)

erase temporary columns

In [25]:
del crash['missing_suburb']
del crash['missing_postcode']


Fornmat crasd_date as time

In [26]:
crash.crash_date=crash.crash_date.astype('datetime64[ns]')

check for duplicated values

In [27]:
crash.duplicated().any()

False

columns with little information, I would consider deleting them. 

In [28]:
crash.contributing_factor_vehicle_5.isnull().sum()/crash.shape[0]

0.98

In [69]:
crash.contributing_factor_vehicle_4.isnull().sum()/crash.shape[0]

0.96

In [70]:
crash.contributing_factor_vehicle_3.isnull().sum()/crash.shape[0]

0.9

In [29]:
crash.to_csv("data_100000_output.csv")